In [32]:
from nltk.tokenize import WhitespaceTokenizer
from nltk.corpus import stopwords
from nltk import RSLPStemmer
from nltk import ngrams
from pandas import DataFrame, read_csv
from string import punctuation
from pathlib import Path
import re

### Importa arquivo

In [14]:
news_data_path = Path('../arquivos/news_data_CLS.csv')
news_df = read_csv(news_data_path)

In [16]:
news_df.head()

,code,source,url,title,datetime,raw_text,cls_influence
0,BBAS3,InfoMoney,https://www.infomoney.com.br/mercados/dolar-de...,"Dólar desacelera queda nesta tarde, com mercad...",2003-06-09 12:05:00.000,SÃO PAULO – Diante das melhores perspectivas p...,1
1,BBAS3,InfoMoney,https://www.infomoney.com.br/mercados/vencimen...,Vencimentos cambiais e ação do Banco do Brasil...,2003-11-14 13:50:00.000,SÃO PAULO – O dólar fechou em alta nesta sexta...,-1
2,BBAS3,InfoMoney,https://www.infomoney.com.br/mercados/banco-do...,Banco do Brasil reduz juros na antecipação da ...,2004-03-30 09:10:00.000,São Paulo – O Banco do Brasil anunciou que a p...,1
3,BBAS3,InfoMoney,https://www.infomoney.com.br/mercados/diretori...,Diretoria de Microcrédito do Banco do Brasil j...,2004-05-25 08:59:00.000,SÃO PAULO – O Banco do Brasil fortaleceu sua a...,1
4,BBAS3,InfoMoney,https://www.infomoney.com.br/mercados/banco-do...,Banco do Brasil estreita relações com micro e ...,2004-06-29 14:50:00.000,SÃO PAULO – O Banco do Brasil (BB) planeja est...,1


In [52]:
news_df['raw_text'][0]

'SÃO PAULO – Diante das melhores perspectivas para a economia brasileira, o dólar comercial segue em queda na tarde desta segunda-feira, com os investidores apostando em novas captações de bancos e empresas brasileiras no mercado externo. Os investidores também seguem atentos aos rumores de uma nova emissão soberana de bônus brasileiros pelo Tesouro Nacional. De acordo com os rumores que começaram a se espalhar pelo mercado na última sexta-feira, o Tesouro estaria preparando uma nova emissão entre US$ 1,0 bilhão e US$ 2,0 bilhões para os próximos dias. Apesar de manter-se em campo negativo, a moeda norte-americana vem desacelerando sua queda no decorrer desta tarde, com os investidores atentos ao comportamento do Banco do Brasil. Na última sexta-feira, rumores davam conta que o BB havia comprado uma grande quantidade de dólares, orientado pelos interesses do Banco Central. Dessa forma, alguns investidores ampliaram as compras de dólares nesta tarde, aproveitando-se também do baixo preç

### Define funções para o tratamento dos textos

In [25]:
def tokenize(text: str) -> list:
    """ Divide the text into token """
    toknzr = WhitespaceTokenizer()
    return toknzr.tokenize(text)

In [21]:
def remove_pt_stop_words(text: str) -> str:
    """ remove stop words from text in portuguese"""
    stop_words = stopwords.words("portuguese")
    no_stop_text = [word for word in text.split() if word not in stop_words]

    return ' '.join(no_stop_text)

In [74]:
def remove_special_chars(text: str) -> str:
    """ Remove punctuation from text """
    clean_text = ' '.join(re.findall('[\w&$]+', text))
    clean_text = re.sub('[\d]|R\$|US\$|\$', ' ', clean_text)
    return clean_text

In [24]:
def stemmize(tokens: list) -> list:
    stemmer = RSLPStemmer()
    return [stemmer.stem(tok) for tok in tokens]

### Aplicação das funções no DataFrame

In [75]:
news_df['prep_text'] = news_df['raw_text'].apply(remove_special_chars)

In [76]:
news_df['prep_text'] = news_df['prep_text'].apply(remove_pt_stop_words)

In [77]:
news_df['prep_text'] = news_df['prep_text'].apply(tokenize)

In [78]:
news_df['prep_text'] = news_df['prep_text'].apply(stemmize)

In [79]:
news_df['prep_text'] = news_df['prep_text'].apply(lambda x : ' '.join(x))

In [80]:
news_df.prep_text[0]

'são paul di melhor perspec econom brasil dól comerc seg qued tard dest segund feir invest apost nov capt banc empr brasil merc extern os invest segu atent rum nov emiss sober bônu brasil tesour nacion de acord rum começ espalh merc últ sext feir tesour est prepar nov emiss bilhã bilhã próx dia apes mant camp nega moed nort americ vem desaceler qued decorr dest tard invest atent comport banc brasil na últ sext feir rum dav cont bb hav compr grand quant dól orient inter banc centr dess form algum invest ampli compr dól nest tard aproveit baix preç relat moed fat condic dess man men qued dól o dól comerc oper cot compr vend baix relaç fech anteri no merc paralel moed nort americ oper negoci represent ági relaç dól comerc com qued dól acumul desvalor junh frent alt registr mê pass no ano desvalor acumul moed nort americ cheg dól futur bm&f na bm&f contrat futur venc julh oper cot lev baix relaç fech últ sext feir o contrat venc agost vez oper fort alt ating frent fech sext feir'

In [81]:
news_df.to_csv(Path('../arquivos/news_data_prep.csv'), index=False)